1. Create `QUALTRICS` folder and put your exported qualtrics csv's inside
2. Create `PROLIFIC` folder and put your exported prolific csv's inside
3. Create `dataframes` folder to save your exported dataframes into.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [ ]:
COMPLETION_CODE_ACCEPTED = ""
COMPLETION_CODE_FAILED_COMPREHENSION = ""
COMPLETION_CODE_REJECTED = ""

In [ ]:
pathlist = Path("./QUALTRICS/").glob('**/*.csv')
dfs = []
for path in pathlist:
    path_in_str = str(path)
    dfs.append(pd.read_csv(path_in_str, skiprows=[1,2]))
df = pd.concat(dfs)
df.drop_duplicates(inplace=True)

In [ ]:
pathlist = Path("./PROLIFIC/").glob('**/*.csv')
dfs = []
for path in sorted(pathlist, key=os.path.getmtime, reverse=True):
    path_in_str = str(path)
    tmp = pd.read_csv(path_in_str)
    tmp["Prolific_File"] = path_in_str.split("\\")[1]
    dfs.append(tmp)
df_prolific = pd.concat(dfs)
df_prolific = df_prolific.rename(columns={"Participant id": "PID"})
print(f'Remove duplicates - len before:  {len(df_prolific)}')
df_prolific = df_prolific.groupby(['PID']).head(1)
print(f'Remove duplicates - len after:  {len(df_prolific)}')

In [ ]:
lstProlificPID = df_prolific[df_prolific.Status == "REJECTED"].PID.values

print(df_prolific.Status.value_counts())
df_prolific = df_prolific[df_prolific.Status != "REJECTED"]

print(f'Amount of other completion code people. {len(df_prolific[df_prolific["Completion code"] != COMPLETION_CODE_ACCEPTED])}')
print('')
df_prolific = df_prolific[df_prolific["Completion code"] == COMPLETION_CODE_ACCEPTED]

print(f'Amount of RETURNED people {len(df_prolific[df_prolific.Status == "RETURNED"])}')
df_prolific = df_prolific[df_prolific.Status != "RETURNED"]

# Merge Qualtrics and Prolific Dataframes

In [ ]:
len(df_prolific.PID.unique()), len(df.PID.unique())

In [ ]:
df = pd.merge(df_prolific, df, on="PID", how="inner")
print(len(df))
df.head()

# AttentionCheck

In [ ]:
ATTENTION_CHECK_COLUMN_NAMES = [] # Add the column names of the attention check here


df_attention = df.copy()
# Change the following row to filter for your attention checks
df_attention = df_attention[(df_attention[ATTENTION_CHECK_COLUMN_NAMES[0]] != "Strongly Agree") | (df_attention[ATTENTION_CHECK_COLUMN_NAMES[1]] <= 92)] #

df_attention = df_attention[['PID', 'AttentionCheckStronglyAgree', 'AttentionCheckRight', "Scenario_ID", "recap", "Prolific_File"]]
df_attention.drop_duplicates(inplace=True)
failedAttentionCheckPIDs = df_attention.PID.values
print(f'Rejection Count: {len(df_prolific[df_prolific.PID.isin(failedAttentionCheckPIDs)])}')

failedAttentionCheckPIDs

In [ ]:
# Filter failed people out
df = df[~df.PID.isin(failedAttentionCheckCount)]

In [ ]:
df.to_csv("./dataframes/00-filtered-dataframe.csv", index=False)